# Team Ernst Engel Food Demand Notebook
In this notebook, we examine a system of demands for various food products, and examine heterogeneity in household consumption.

In [1]:
import pandas as pd
import numpy as np
from cfe import Regression
import warnings

In [2]:
%run nutritional_adequacy.ipynb

## Data Processing

In [4]:
# food_nutrients, food_prices, hh_chars, expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs = get_data(country, year_range)

In [6]:
# warnings.filterwarnings("ignore")
# x = expenditures
# x.columns.name = 'j'
# x = x.T.groupby('j').sum().T
# x = x.replace(0,np.nan)

# y = np.log(x.set_index(['i','t','m']))

# p = fp_sub
# p.columns.name = 'j'

# d = hhc_sub.copy()
# d.columns.name = 'k'
# d.set_index(['i','t','m'],inplace=True)

In [10]:
def process_data(country, year):
    food_nutrients, food_prices, hh_chars, expenditures, hhc_sub, fp_sub, food_cols, fp_sub_avgs = get_data(country, year)
    warnings.filterwarnings("ignore")
    x = expenditures
    x.columns.name = 'j'
    x = x.T.groupby('j').sum().T
    x = x.replace(0,np.nan)

    y = np.log(x.set_index(['i','t','m']))

    p = fp_sub
    p.columns.name = 'j'

    d = hhc_sub.copy()
    d.columns.name = 'k'
    d.set_index(['i','t','m'],inplace=True)
    
    y = y.stack()

    d = d.stack()
    
    return x, y, p, d

In [12]:
x, y, p, d = process_data(country, year_range)

## Estimation
Recall the model put forth in lecture: <br>
Let $y_{i}^j$ be log expenditures on food $j$ by household $i$ at a particular time.  We want to estimate a regression that takes the form
$$
      y^j_{i} = A^j(p) + \gamma_j'd_i + \beta_j w_i + \zeta^j_i.
$$

In [15]:
# result = Regression(y = y,d = d)

In [17]:
# plt.figure(figsize=(9.25, 6))
# plot_df = pd.DataFrame({'y' : y,'yhat' : result.get_predicted_log_expenditures()})

# sns.scatterplot(data = plot_df, x = 'yhat', y = 'y', alpha = 0.2)

# dummy_x = np.linspace(4, 12, 77)

# ### Overlay y = x line
# plt.plot(dummy_x, dummy_x, color='red', label = r'$y = \hat{y}$')

# plt.xlabel(r'Predicted Expenditures')
# plt.ylabel(r'Real Expenditures')
# plt.title('Real vs. Predicted Expenditures')
# plt.legend()

# plt.show()

## Parameters

### (Relative) Income Elasticity

In [24]:
# ax = result.graph_beta();
# plt.title('Income Elasticity')
# plt.show();

### Welfare

In [27]:
# plt.figure(figsize=(9.25, 6))
# ax = result.get_w().plot.hist(bins = 100, density = True)
# result.get_w().plot.kde(ax=ax)
# plt.axvline(x = 0, color = 'black', linestyle = '--')
# plt.ylabel(r'$p(w_{i})$')
# plt.xlabel(r'$w_{i}$')
# plt.title('Household Welfare Distribution')
# plt.show();

In [29]:
def run_estimation(y, d):
    result = Regression(y = y,d = d)
    
    ### Plots
    plt.figure(figsize=(9.25, 6))
    plot_df = pd.DataFrame({'y' : y,'yhat' : result.get_predicted_log_expenditures()})

    sns.scatterplot(data = plot_df, x = 'yhat', y = 'y', alpha = 0.2)

    dummy_x = np.linspace(4, 12, 77)

    ### Overlay y = x line
    plt.plot(dummy_x, dummy_x, color='red', label = r'$y = \hat{y}$')

    plt.xlabel(r'Predicted Expenditures')
    plt.ylabel(r'Real Expenditures')
    plt.title('Real vs. Predicted Expenditures')
    plt.legend()
    plt.show();
    
    ax = result.graph_beta();
    plt.title('Income Elasticity')
    plt.show();
    
    plt.figure(figsize=(9.25, 6))
    ax = result.get_w().plot.hist(bins = 100, density = True)
    result.get_w().plot.kde(ax=ax)
    plt.axvline(x = 0, color = 'black', linestyle = '--')
    plt.ylabel(r'$p(w_{i})$')
    plt.xlabel(r'$w_{i}$')
    plt.title('Household Welfare Distribution')
    plt.show();
    
    return result

## Demand and Utility (Counterfactual Experiment)

### Budgets

In [37]:
# xhat = result.predicted_expenditures()

# xbar = xhat.groupby(['i','t','m']).sum()

# ### Reference budget
# xref = xbar.quantile(0.5)  # 0.5 ==> median

### Reference Prices

In [158]:
# ### Prices per kilogram:
# pbar = p.mean()
# pbar = pbar[result.beta.index]

def change_price(p0, p, j = 'Millet'):
    """
    Change price of jth good to p0, holding other prices fixed.
    """
    p = p.copy()
    p.loc[j] = p0
    return p

In [160]:
def plot_demands(food_product, result, p):
    xhat = result.predicted_expenditures()

    xbar = xhat.groupby(['i','t','m']).sum()

    ### Reference budget
    xref = xbar.quantile(0.5)  # 0.5 ==> median
    
    ### Prices per kilogram:
    pbar = p.mean()
    pbar = pbar[result.beta.index]
    
    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)

    # Demandfor household at median budget
    plt.plot([result.demands(xref, change_price(pbar[food_product] * s, pbar, food_product))[food_product] for s in scale], 
             scale, label = 'Median')

    # Demand for household at 25% percentile
    plt.plot([result.demands(xbar.quantile(0.25), change_price(pbar[food_product] * s, pbar, food_product))[food_product] for s in scale], 
             scale, label = '25th Percentile')

    # Demand for household at 75% percentile
    plt.plot([result.demands(xbar.quantile(0.75), change_price(pbar[food_product] * s, pbar, food_product))[food_product] for s in scale], 
             scale, label = '75th Percentile')

    plt.title(f"Demand of {food_product}")
    plt.ylabel(f"Price (relative to base of {pbar[food_product]:.2f})")
    plt.xlabel(f"Quantity")
    plt.legend()
    plt.show();

In [152]:
# plot_demands(food_product, result)

### Engel Curves

In [154]:
# desired_foods = ['Beans', 'Bread', 'Millet']
# fig,ax = plt.subplots()

# # Vary prices from 50% to 200% of reference.
# scale = np.linspace(.5,2,20)
# # ax.plot(np.log(scale * xref),[(result.expenditures(s * xref, pbar)/(s * xref)).loc[desired_foods] for s in scale])

# ax.plot(np.log(scale * xref),[result.expenditures(s * xref, pbar)/(s * xref) for s in scale])

# ax.set_xlabel(f'log budget (relative to base of {xref:.0f}')
# ax.set_ylabel(f'Expenditure share')
# ax.set_title('Engel Curves')
# plt.show();

In [143]:
def plot_engel_curve(xref, result, p, desired_foods = ['all']):
    fig,ax = plt.subplots()
    
    xhat = result.predicted_expenditures()

    xbar = xhat.groupby(['i','t','m']).sum()

    ### Reference budget
    xref = xbar.quantile(0.5)  # 0.5 ==> median
    
    ### Prices per kilogram:
    pbar = p.mean()
    pbar = pbar[result.beta.index]

    # Vary prices from 50% to 200% of reference.
    scale = np.linspace(.5,2,20)
    try:
        ax.plot(np.log(scale * xref),
                [(result.expenditures(s * xref, pbar)/(s * xref)).loc[desired_foods] for s in scale])
        
    except:
        ax.plot(np.log(scale * xref),[result.expenditures(s * xref, pbar)/(s * xref) for s in scale])

    ax.set_xlabel(f'log budget (relative to base of {xref:.0f})')
    ax.set_ylabel(f'Expenditure Share')
    ax.set_title('Engel Curves')
    plt.show();  

In [156]:
# plot_engel_curve(xref)

### Indirect Utility

In [178]:
# fig,ax = plt.subplots()

# scale = np.linspace(.5,2,20)
# ax.plot(scale*xref,[result.indirect_utility(s*xref,pbar) for s in scale])
# ax.set_xlabel(f'Indirect Utility (Budget relative to base of {xref:.0f}')
# ax.set_ylabel(f'Utility')
# ax.set_title('Indirect Utility Function')
# plt.show();

In [191]:
def plot_indirect_utility(result, p):
    
    xhat = result.predicted_expenditures()

    xbar = xhat.groupby(['i','t','m']).sum()

    ### Reference budget
    xref = xbar.quantile(0.5)  # 0.5 ==> median
    
    ### Prices per kilogram:
    pbar = p.mean()
    pbar = pbar[result.beta.index]

    fig,ax = plt.subplots(figsize=(9.25, 6))

    scale = np.linspace(.5,2,20)
    ax.plot(scale*xref,[result.indirect_utility(s*xref,pbar) for s in scale])
    ax.set_xlabel(f'Indirect Utility (Budget relative to base of {xref:.0f}')
    ax.set_ylabel(f'Utility')
    ax.set_title('Indirect Utility Function')
    plt.show();

## Interactivity Development

In [168]:
def estimate_demand_wrapper(country, year):
    x, y, p, d = process_data(country, year_range)
    result = run_estimation(y, d)
    plot_indirect_utility(result, p)
    return result, p

In [196]:
def plot_demands_interactive(result, p):
    widget = interactive(plot_demands,
                         food_product = result.beta.index,
                         result = widgets.fixed(result),
                        p = widgets.fixed(p))
    display(widget)